<a href="https://www.kaggle.com/code/mikedelong/essay-keywords-first-look?scriptVersionId=153075438" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
! pip install sentence-transformers 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=3a8522523cd3cbb839986a17bf72340d3027546f248d8ed32fe7dab5577031f9
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [2]:
with open(file='/kaggle/input/paul-graham-essays/paul_graham_essays.txt', encoding='utf-8', mode='r') as input_fp:
    text = input_fp.read()
essays = text.split('\n')
print('essay count: {}'.format(len(essays)))

essay count: 55


In [3]:
# https://www.kaggle.com/code/ianalyticsgeek/keywords-extraction-using-bert
from arrow import now
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer

# we can't use more features than we can visualize
MAX_FEATURES = 600

NGRAM_RANGE = (1, 1)
model_start = now()
count = TfidfVectorizer(ngram_range=NGRAM_RANGE, stop_words='english', max_features=MAX_FEATURES).fit(essays)
features = count.get_feature_names_out()
print('{}: got features'.format(now()))
model = SentenceTransformer('distilbert-base-nli-mean-tokens')
embedding = model.encode(essays)
print('{}: got embeddings'.format(now()))
feature_embeddings = model.encode(features)
print('{}: got feature embeddings'.format(now()))
print('model time: {}'.format(now() - model_start))

2023-11-30T20:01:05.569798+00:00: got features


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

2023-11-30T20:01:20.904862+00:00: got embeddings


Batches:   0%|          | 0/19 [00:00<?, ?it/s]

2023-11-30T20:01:23.220013+00:00: got feature embeddings
model time: 0:00:18.306672


In [4]:
# post process the features 
keep = set(features)
# filter out numeric values
keep = {item for item in keep if not item.isnumeric()}
# do some basic plural identification and remove the plurals 
plurals = {item for item in keep if (item.endswith('s') and item[:-1] in keep) or (item.endswith('es') and item[:-2] in keep)}
keep = keep.difference(plurals)
print(len(keep), keep)

534 {'ll', 'simple', 'used', 'wisdom', 'patent', 'simply', 'internet', 'main', 'chance', 'way', 'rich', 'mean', 'machine', 'fairly', 'powerful', 'harder', 'working', 'guess', 'especially', 'feel', 'knew', 'language', 'win', 'mistake', 'mistaken', 'does', 'yahoo', 'series', 'famous', 'created', 'recently', 'say', 'java', 'haven', 'hardware', 'probably', 'sense', 'yc', 'partly', 'future', 'happen', 'short', 'idea', 'buy', 'tell', 'technology', 'spent', 'started', 'independent', 'single', 'programming', 'line', 'rate', 'children', 'saying', 'scale', 'hope', 'year', 'quite', 'face', 'school', 'ones', 'fast', 'answer', 'matter', 'step', 'like', 'math', 'create', 'weren', 'high', 'old', 'certainly', 'hacking', 'invest', 'practically', 'predict', 'solution', 'rule', 'surprised', 'opinions', 'form', 'bought', 'actually', 'organizations', 'present', 'raise', 'round', 'pay', 'point', 'starting', 'lot', 'word', 'book', 'doing', 'built', 'path', 'cause', 'million', 'performance', 'start', 'month',

In [5]:
from nltk import pos_tag
from nltk import word_tokenize

noun_start = now()
keep_nouns = {word for item in essays for (word, part_of_speech) in pos_tag(word_tokenize(item)) if part_of_speech[:2] == 'NN' and word in keep}
print('filtered nouns in {}'.format(now() - noun_start))
keep = keep_nouns
print(len(keep), keep)

filtered nouns in 0:00:41.900997
348 {'simple', 'wisdom', 'patent', 'internet', 'chance', 'way', 'rich', 'mean', 'machine', 'harder', 'working', 'guess', 'knew', 'feel', 'language', 'win', 'mistake', 'mistaken', 'series', 'famous', 'say', 'haven', 'hardware', 'sense', 'short', 'future', 'happen', 'idea', 'buy', 'tell', 'technology', 'spent', 'programming', 'line', 'rate', 'children', 'saying', 'scale', 'year', 'hope', 'face', 'school', 'ones', 'fast', 'answer', 'step', 'math', 'like', 'create', 'hacking', 'invest', 'predict', 'solution', 'rule', 'opinions', 'form', 'organizations', 'bought', 'raise', 'present', 'round', 'pay', 'point', 'lot', 'word', 'book', 'path', 'cause', 'performance', 'start', 'month', 'situation', 'lose', 'notes', 'reason', 'car', 'web', 'adults', 'silicon', 'code', 'university', 'stop', 'essay', 'class', 'pick', 'living', 'economy', 'quality', 'source', 'online', 'want', 'knowledge', 'hand', 'world', 'search', 'theory', 'tend', 'selling', 'paul', 'measure', 'cus

In [6]:
# now we can project the feature vectors into 2-space to visualize 
from pandas import DataFrame
from plotly.express import scatter
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=2023, verbose=1, n_iter=10000)
tsne_df = DataFrame(data=tsne.fit_transform(X=feature_embeddings), columns=['t0', 't1'])
tsne_df['word'] = features
scatter(data_frame=tsne_df[tsne_df['word'].isin(keep)], x='t0', y='t1', text='word', height=900, ).update_traces(marker={'size': 1})

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 600 samples in 0.001s...
[t-SNE] Computed neighbors for 600 samples in 0.186s...
[t-SNE] Computed conditional probabilities for sample 600 / 600
[t-SNE] Mean sigma: 0.000000
[t-SNE] KL divergence after 250 iterations with early exaggeration: 51.252754
[t-SNE] KL divergence after 1400 iterations: 1.320877


In [7]:
{item for item in keep if item.endswith('ing')}

{'asking',
 'building',
 'funding',
 'fundraising',
 'hacking',
 'interesting',
 'investing',
 'learning',
 'living',
 'making',
 'painting',
 'programming',
 'promising',
 'reading',
 'running',
 'saying',
 'selling',
 'surprising',
 'thing',
 'thinking',
 'working',
 'writing'}